In [1]:
import numpy as np
import pandas as pd
import random
import os

import torch
from torch import nn
# Import necessary libraries
from onn.OnlineNeuralNetwork import ONN
from tqdm import tqdm

from river import preprocessing
from D3 import D3
try:
    from skmultiflow.data.data_stream import DataStream
except:
    from skmultiflow.data.data_stream import DataStream
from sklearn.metrics import accuracy_score, f1_score, classification_report

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

from scipy.stats import entropy
from sklearn.cluster import KMeans
warnings.filterwarnings('ignore')


In [2]:
def set_env(seed):
    torch.cuda.set_device(0)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = True
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    # os.environ['OMP_NUM_THREADS'] = '1'
    
    
def prequential_eval(test_y, y_pred):
    chunk_num = 30
    chunk_size = len(test_y) // chunk_num

    acc_list = []
    f1_list = []

    for i in range(chunk_num):
        start_idx = i*chunk_size
        end_idx = (i+1)*chunk_size
        if i == (chunk_num-1):
            chunk_y = test_y[start_idx:]
            chunk_pred = y_pred[start_idx:]
        else:
            chunk_y = test_y[start_idx:end_idx]
            chunk_pred = y_pred[start_idx:end_idx]

        acc_list.append(accuracy_score(chunk_y, chunk_pred))
        f1_list.append(f1_score(chunk_y, chunk_pred))

    return acc_list, f1_list

def plot_acc_f1(acc_list, f1_list):
    # 1행 2열의 subplot 생성
    fig, ax = plt.subplots(1, 2, figsize=(12, 5))

    # 첫 번째 subplot에 accuracy 데이터 플로팅
    ax[0].plot(range(30), acc_list, marker='o', color='b', linestyle='-', linewidth=2, markersize=5)
    ax[0].set_xlabel('Percentage of data')
    ax[0].set_ylabel('Accuracy')
    ax[0].grid(True)
    ax[0].set_title('Accuracy Plot')

    # 두 번째 subplot에 F1 Score 데이터 플로팅
    ax[1].plot(range(30), f1_list, marker='o', color='b', linestyle='-', linewidth=2, markersize=5)
    ax[1].set_xlabel('Percentage of data')
    ax[1].set_ylabel('F1 Score')
    ax[1].grid(True)
    ax[1].set_title('F1 Score Plot')

    # 그래프 보여주기
    plt.tight_layout()
    plt.show()

# offline online 비교 코드
def plot_acc_f1_together(offline_acc_list, offline_f1_list, online_acc_list, online_f1_list):
    # 1행 2열의 subplot 생성
    fig, ax = plt.subplots(1, 2, figsize=(12, 5))

    # 첫 번째 subplot에 Accuracy 데이터 플로팅
    ax[0].plot(range(30), offline_acc_list, marker='o', color='b', linestyle='-', linewidth=2, markersize=5, label="Offline")
    ax[0].plot(range(30), online_acc_list, marker='o', color='r', linestyle='-', linewidth=2, markersize=5, label="Online")
    ax[0].set_xlabel('Percentage of data')
    ax[0].set_ylabel('Accuracy')
    ax[0].grid(True)
    ax[0].set_title('Accuracy Plot')
    ax[0].legend()

    # 두 번째 subplot에 F1 Score 데이터 플로팅
    ax[1].plot(range(30), offline_f1_list, marker='o', color='b', linestyle='-', linewidth=2, markersize=5, label="Offline")
    ax[1].plot(range(30), online_f1_list, marker='o', color='r', linestyle='-', linewidth=2, markersize=5, label="Online")
    ax[1].set_xlabel('Percentage of data')
    ax[1].set_ylabel('F1 Score')
    ax[1].grid(True)
    ax[1].set_title('F1 Score Plot')
    ax[1].legend()

    # 그래프 보여주기
    plt.tight_layout()
    plt.show()

## Active learning

In [3]:
import pickle

def uncertainty_measure(model, X):
    probas = model.predict_proba(X)[0]
    uncertainty = entropy(probas)  # entropy 값
    return uncertainty

def save_result_chunk(test_y, pred_li, file_name):
    test_y = test_y.values
    chunk_num = 30
    chunk_size = len(test_y) // chunk_num

    f1_list = []

    for i in range(chunk_num):
        start_idx = i*chunk_size
        end_idx = (i+1)*chunk_size
        if i == (chunk_num-1):
            chunk_y = test_y[start_idx:]
            chunk_pred = pred_li[start_idx:]
        else:
            chunk_y = test_y[start_idx:end_idx]
            chunk_pred = pred_li[start_idx:end_idx]

        f1_list.append(f1_score(chunk_y, chunk_pred))

    with open(f"active_learning/{file_name}.pkl", "wb") as f:
        pickle.dump(f1_list, f)

In [7]:
df = pd.read_csv('경진대회용 주조 공정최적화 데이터셋.csv', encoding='cp949', index_col=0)
df.dropna(subset=['emergency_stop', 'upper_mold_temp3','lower_mold_temp3'], inplace=True)
df = df[['registration_time','facility_operation_cycleTime','production_cycletime', 'low_section_speed', 'high_section_speed','cast_pressure', 'biscuit_thickness',
        'upper_mold_temp1', 'upper_mold_temp2', 'upper_mold_temp3','lower_mold_temp1', 'lower_mold_temp2', 'lower_mold_temp3','sleeve_temperature', 'physical_strength', 'Coolant_temperature',
        'EMS_operation_time','passorfail']]
df['registration_time'] = pd.to_datetime(df['registration_time'])

train_df = df[df['registration_time'] <= '2019-01-30 08:00:00']
test_df = df[df['registration_time'] > '2019-01-30 08:00:00']
train_X = train_df.drop(['registration_time','passorfail'],axis=1)
train_y = train_df['passorfail']
test_X = test_df.drop(['registration_time','passorfail'],axis=1)
test_y = test_df['passorfail']

result_df = pd.DataFrame()
result_df['y_true'] = test_y
result_df.to_csv('active_learning_result/result_active_learning.csv', index=False)

In [ ]:
import pickle
def run_AL(method, alpha):
    set_env(42)
    
    print(f"📍 Start for {method} {alpha}")
    result_df = pd.read_csv('active_learning_result/result_active_learning.csv', index_col=False)
    df = pd.read_csv('경진대회용 주조 공정최적화 데이터셋.csv', encoding='cp949', index_col=0)
    df.dropna(subset=['emergency_stop', 'upper_mold_temp3','lower_mold_temp3'], inplace=True)
    df = df[['registration_time','facility_operation_cycleTime','production_cycletime', 'low_section_speed', 'high_section_speed','cast_pressure', 'biscuit_thickness',
            'upper_mold_temp1', 'upper_mold_temp2', 'upper_mold_temp3','lower_mold_temp1', 'lower_mold_temp2', 'lower_mold_temp3','sleeve_temperature', 'physical_strength', 'Coolant_temperature',
            'EMS_operation_time','passorfail']]
    df['registration_time'] = pd.to_datetime(df['registration_time'])

    train_df = df[df['registration_time'] <= '2019-01-30 08:00:00']
    test_df = df[df['registration_time'] > '2019-01-30 08:00:00']
    train_X = train_df.drop(['registration_time','passorfail'],axis=1)
    train_y = train_df['passorfail']
    test_X = test_df.drop(['registration_time','passorfail'],axis=1)
    test_y = test_df['passorfail']
    
    # 기존 컬럼 이름을 새 이름으로 매핑하는 딕셔너리 생성
    new_column_names = {train_X.columns[i]: i for i in range(16)}

    # 컬럼 이름 변경
    train_X = train_X.rename(columns=new_column_names)
    test_X = test_X.rename(columns=new_column_names)

    print('train shape: ', train_X.shape, 'test shape: ', test_X.shape)
    
    # 모델 저장 경로
    save_path = "model_params/Offline_onn_params.pth"

    # 저장한 가중치 불러오기
    state_dict = torch.load(save_path)

    # 새로운 모델 객체 초기화
    onn_network = ONN(features_size=train_X.shape[1], 
                    max_num_hidden_layers=2, 
                    qtd_neuron_per_hidden_layer=10, 
                    n_classes=train_y.nunique(),
                    use_cuda=True
                    )

    # 모델에 저장된 가중치를 불러오기 (strict=False)
    missing_keys, unexpected_keys = onn_network.load_state_dict(state_dict, strict=False)
    # alpha 파라미터 수동 설정
    onn_network.alpha = state_dict["alpha"].to(onn_network.device)
    onn_network.to(onn_network.device)
    
    # 1. 모델 및 스케일러 초기화
    scaler = preprocessing.StandardScaler()

    # 2. Train 데이터 스케일링 후 TabNet 모델 학습
    scaler.learn_many(train_X) # train_X 전체 스케일링
    train_X_scaled = scaler.transform_many(train_X).values
        
    pred_li = []
    i = 0
    w_size = 500
    rho = 0.4

    stream = DataStream(data = test_X, y= test_y.values)
    d3 = D3(w=w_size, rho=rho, dim = 16, auc = 0.85)

    # Drift Detector의 초기 데이터를 채움
    for x, y in zip(train_X.values[-w_size:, :], train_y.values[-w_size:]):
        d3.addInstance(x, y)               

    # Prequential Learning Loop
    while stream.has_more_samples():
        X, y = stream.next_sample()
        # 1. 스케일링
        X_scaled = scaler.transform_one(dict(enumerate(X[0])))  # Test 샘플을 스케일링
        X_scaled = np.array([list(X_scaled.values())])  # dict_values를 list로 변환 후 numpy array로 변환
        
        # 2. 예측 후 저장
        y_hat = onn_network.predict(X_scaled)
        pred_li.append(y_hat.item())

        if d3.isEmpty():  # D3 Detector가 비어 있는 경우
            d3.addInstance(X, y) 
        else:
            if d3.driftCheck():  # Drift가 감지된 경우
                print(f"Concept drift detected at sample {i}")

                if method == 'random':

                    n = len(d3.getCurrentData())
                    idx = np.random.choice(np.arange(n), size=int(n * alpha), replace=False)
                    idx.sort()
                    sampled_data = d3.getCurrentData()[idx]
                    sampled_labels = d3.getCurrentLabels()[idx]

                if method == 'entropy':
                    # active learning 으로 라벨링 데이터 선별
                    uncertain_data, uncertain_labels = [], []
                    uncertainty = []
                    for instance, label in zip(d3.getCurrentData(), d3.getCurrentLabels()):  # 200개 중
                        uncertainty_value = uncertainty_measure(onn_network, instance.reshape(1, -1))
                        uncertainty.append(uncertainty_value)

                    n = len(d3.getCurrentData())

                    idx = np.argsort(uncertainty)[-int(n*alpha):][::-1]  # 내림차순으로 정렬된 상위 n개 인덱스
                    idx.sort()
                    sampled_data = d3.getCurrentData()[idx]
                    sampled_labels = d3.getCurrentLabels()[idx]
                
                if method == 'kmeans':

                    # K-means clustering을 통해 데이터 클러스터링
                    k = int(len(d3.getCurrentData()) * alpha)
                    kmeans = KMeans(n_clusters=k, random_state=0)
                    data = d3.getCurrentData()  # 현재 데이터 버퍼의 데이터들
                    kmeans.fit(data)

                    # 각 클러스터의 중심점에서 가장 가까운 데이터 선택
                    sampled_data, sampled_labels = [], []
                    for cluster_idx in range(k):
                        # 각 클러스터의 모든 데이터 인덱스 가져오기
                        cluster_data_indices = np.where(kmeans.labels_ == cluster_idx)[0]
                        cluster_data = data[cluster_data_indices]

                        # 클러스터 중심과 각 데이터 포인트 간 거리 계산
                        center = kmeans.cluster_centers_[cluster_idx]
                        distances = np.linalg.norm(cluster_data - center, axis=1)

                        # 중심에 가장 가까운 데이터 선택
                        closest_index = cluster_data_indices[np.argmin(distances)]
                        sampled_data.append(data[closest_index])
                        sampled_labels.append(d3.getCurrentLabels()[closest_index])

                
                scaler.learn_many(pd.DataFrame(sampled_data)) # 새로운 데이터 스케일링
                drift_X_scaled = scaler.transform_many(pd.DataFrame(sampled_data)).values


                # 모델 미세 조정
                for j in tqdm(range(len(drift_X_scaled))):
                    onn_network.partial_fit(np.asarray([drift_X_scaled[j]]), np.asarray([sampled_labels[j]]),show_loss=False)
                    
                d3.addInstance(X, y)  # Drift 후 데이터 추가
            else:
                d3.addInstance(X, y)  # Drift가 감지되지 않은 경우 데이터 추가
        
        i += 1
    result_df[f'{method}_{alpha}'] = pred_li
    result_df.to_csv('active_learning_result/result_active_learning.csv', index=False)

    save_result_chunk(test_y, pred_li, f'F1_{method}_{alpha}')  # f1 저장

    accuracy = accuracy_score(test_y, pred_li)
    f1 = f1_score(test_y, pred_li)
    report = classification_report(test_y, pred_li)

    print(f"Accuracy({method}({alpha})):", accuracy)
    print(f"F1 Score({method}({alpha})):", f1)
    print(f"Classification Report({method}({alpha})):\n", report)

In [6]:
for method in ['random', 'entropy', 'kmeans']:
    for alpha in [0.2, 0.4, 0.6, 0.8]:
        run_AL(method, alpha)

📍 Start for random 0.2
train shape:  (26461, 16) test shape:  (65241, 16)
Using CUDA :]
Concept drift detected at sample 200


100%|██████████| 40/40 [00:00<00:00, 724.02it/s]


Concept drift detected at sample 1300


100%|██████████| 40/40 [00:00<00:00, 1247.29it/s]


Concept drift detected at sample 2000


...

Concept drift detected at sample 64700


100%|██████████| 160/160 [00:00<00:00, 1486.21it/s]


Concept drift detected at sample 65200


100%|██████████| 160/160 [00:00<00:00, 1481.77it/s]


Accuracy(kmeans(0.8)): 0.9826949311016079
F1 Score(kmeans(0.8)): 0.7796213156353698
Classification Report(kmeans(0.8)):
               precision    recall  f1-score   support

         0.0       0.99      1.00      0.99     62354
         1.0       0.89      0.69      0.78      2887

    accuracy                           0.98     65241
   macro avg       0.94      0.84      0.89     65241
weighted avg       0.98      0.98      0.98     65241

